In [1]:
import pandas as pd
import seaborn as sns

sns.set_theme(style="darkgrid")
sns.set_palette("Set2")

In [2]:
# Cell that reading Log file
logfile = open('scalability.log', 'r')
lines = logfile.readlines()

# Parse experiment config
loop_count = int(lines[0])
client_start = int(lines[1])
max_clients = int(lines[2])
iterations = int(lines[3])

# Preparing columns
run_numbers = []
client_ids = []
latency = []
num_cur_clients = []
approaches = []

ct = 4
# Parse experiment results
for num_clients in range(client_start, max_clients+1):
    for i in range (loop_count):
        for id in range(0, num_clients):
            run_numbers.append(i)
            client_ids.append(id)
            latency.append(float(lines[ct]))
            num_cur_clients.append(num_clients)
            approaches.append("independent unelevated")
            ct += 1
    for i in range (loop_count):
        for id in range(0, num_clients):
            run_numbers.append(i)
            client_ids.append(id)
            latency.append(float(lines[ct]))
            num_cur_clients.append(num_clients)
            approaches.append("server client unelevated")
            ct += 1

"""
for i in range (loop_count):
    for id in range(0, max_clients+1):
        run_numbers.append(i)
        client_ids.append(id)
        latency.append(float(lines[ct]))
        num_cur_clients.append(max_clients+1)
        approaches.append("independent unelevated")
        ct += 1
"""

# Prepare dataframe

dict = {
    'run#': run_numbers,
    'client_id': client_ids,
    'latency': latency,
    'num clients': num_cur_clients,
    'approach': approaches
}

df = pd.DataFrame(dict)

df['throughput'] = ([iterations]*len(df)) / df['latency']
df.head()

,run#,client_id,latency,num clients,approach,throughput
0,0,0,0.588209,1,independent unelevated,170007.599340
1,1,0,0.581735,1,independent unelevated,171899.576268
2,2,0,0.580953,1,independent unelevated,172130.964123
3,3,0,0.581736,1,independent unelevated,171899.280773
4,4,0,0.581697,1,independent unelevated,171910.805798


In [3]:
p1=sns.lmplot(data=df, x="num clients", y="throughput", hue="approach",  height=10)
p1.set_axis_labels("number of concurrent clients", "throughput (req per second)")

In [4]:
avg_df = df.groupby(["num clients", "approach"])["throughput"].mean().reset_index()
p2=sns.lineplot(data=avg_df, x="num clients", y="throughput", hue="approach")